# БОТ, СОЗДАЮЩИЙ ОБЛАКА СЛОВ

In [1]:
pip install https://github.com/aiogram/aiogram/archive/refs/heads/dev-3.x.zip

Defaulting to user installation because normal site-packages is not writeable
  Using cached https://github.com/aiogram/aiogram/archive/refs/heads/dev-3.x.zip
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nest_asyncio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pymorphy3

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=0375329f0beab32aa1037713c7d1d95489de3ceba2d0c32d2e6139a4c461b953
  Stored in directory: c:\users\klim\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [28]:
import os
import nest_asyncio
import asyncio
import logging
from aiogram import Bot, Dispatcher, types, filters, Router, handlers, F
from aiogram.filters import Command
from aiogram.types import FSInputFile

import json
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation

from pymorphy3 import MorphAnalyzer
pymorphy3_analyzer = MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Klim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
def prepare(infile, switch):
    if switch == 'channel':
        with open(infile, 'r', encoding='utf8') as f:
            jdata = f.read()
        data = json.loads(jdata)
        messages = data['messages']
        df = pd.DataFrame(messages)
        df['text'] = df['text'].apply(lambda x: [i for i in x if type(i) == str] if type(x) == list else x)
        df['text'] = df['text'].astype('str')
        df['text'].replace(r'\\n', ' ', regex=True, inplace=True)
        df['text'].replace(r'\n', ' ', regex=True, inplace=True)
        text = ' '.join(df['text'].to_list()).lower()
    elif switch == 'txt':
        with open(infile, 'r', encoding='utf8') as f:
            text = f.read().lower().replace('\n', ' ')
    return text

def cloud(text, infile):
    text_masked = ' '.join([pymorphy3_analyzer.parse(word)[0].normal_form for word in text.split() if
                            pymorphy3_analyzer.parse(word)[0].tag.POS == 'NOUN'])
    noise = stopwords.words('russian')
    punctuation_new = list(punctuation) + list('»') + list('«')
    wordcloud = WordCloud(stopwords=noise + punctuation_new + ['мочь', 'который', 'свой', 'весь'], 
                          mode='RGBA', background_color=None).generate(text_masked)
    outfile = infile.replace('json', 'png').replace('txt', 'png')
    wordcloud.to_file(outfile)
    return outfile

In [ ]:
nest_asyncio.apply()

logging.basicConfig(level=logging.INFO)

TOKEN = '5930567047:AAFUeBgC1hDCaay14Sr47NjdBU8ZLkNQta0'

async def main():
    bot = Bot(token=TOKEN)
    global switch
    switch = 'off'
    dp = Dispatcher()
    dp.include_router(router)
    await bot.delete_webhook(drop_pending_updates=True)
    await dp.start_polling(bot)

router = Router()

@router.message(Command('start'))
async def start(message): 
    global switch
    switch = 'off'
    kb = [
        [types.KeyboardButton(text='Telegram канал/чат')],
        [types.KeyboardButton(text='Текстовый файл (с расширением .txt)')],
        [types.KeyboardButton(text='Спасибо, не надо облаков.')]
    ]
    keyboard = types.ReplyKeyboardMarkup(keyboard=kb, resize_keyboard=True, one_time_keyboard=True)
    await message.answer_sticker(r'CAACAgUAAxkBAAEGlihjhJzOCae5KywwFe850cZstkjtawACtQUAAqMFQQctHaXKG0pkxysE')
    await message.answer('Здравствуйте.\nМеня зовут Клауди и я делаю облака слов.\n\
Что Вы хотите превратить в облако?', reply_markup=keyboard)

@router.message(lambda message: message.text == 'Telegram канал/чат')
async def option_channel(message):
    global switch
    switch = 'channel'
    await message.answer('Экспортируйте историю канала/чата в формате JSON\
 (данная функция доступна только в Telegram Desktop) и \
вставьте в сообщение ниже.\n\nВведите любое сообщение, чтобы начать заново.')
    
@router.message(lambda message: message.text == 'Текстовый файл (с расширением .txt)')
async def option_txt(message):
    global switch
    switch = 'txt'
    await message.answer('Вставьте текстовый файл в сообщение ниже.\nЧем чище файл, тем красивее облако.\n\n\
Введите любое сообщение, чтобы начать заново.')

@router.message(~F.content_type.in_({'text'}))
async def inout(message, bot):
    if switch in ['channel', 'txt']:
        infile_error = False
        try:
            os.makedirs(f'{os.getcwd()}\\cloudy_files', exist_ok=True)
            if switch == 'channel':
                extension = '.json'
            elif switch == 'txt':
                extension = '.txt' 
            infile = f'{os.getcwd()}\\cloudy_files\\{message.document.file_id}{extension}'
            await bot.download(message.document, destination=infile)
            await message.answer_sticker(r'CAACAgUAAxkBAAEGmAxjhOd0fyuNg24ngDHR_mMZLWeGqQAC-QUAAqMFQQeMBfF0g8QOLisE')
            await message.answer('Делаю облако.\nВам необязательно здесь ждать. Я пришлю результат с вороном.')
            text = prepare(infile, switch)
        except:
            infile_error = True
            await message.answer_sticker(r'CAACAgUAAxkBAAEGn0ljhvkqvM96D6icF-IyO9Zxr5O49wAC5wUAAqMFQQfz3mvY5aG69CsE')
            if switch == 'json':
                await message.answer('Не получается обработать файл.\nВставьте экспорт канала/чата\
 в формате JSON.\n\nВведите любое сообщение, чтобы начать заново.')
            if switch == 'txt':
                await message.answer('Не получается обработать файл.\nВставьте текстовый файл\
 с расширением .txt.\n\nВведите любое сообщение, чтобы начать заново.')
        if infile_error == False:
            try:
                outfile = cloud(text, infile)
                result = FSInputFile(outfile)
                await message.answer_photo(result)
                await message.answer('Ваше облако готово.\nЕсли хотите ещё одно, вставьте новый файл.\n\n\
Введите любое сообщение, чтобы начать заново.')
            except:
                await message.answer_sticker(r'CAACAgUAAxkBAAEGn0tjhvnDHh3GjdC946_BOYClWEZ7BQACuQUAAqMFQQfoxuUFtV3sGisE')
                await message.answer('Что-то сломалось. Попробуйте вставить другой файл в формате JSON.\n\n\
Введите любое сообщение, чтобы начать заново.')
    else:
        kb = [
        [types.KeyboardButton(text='/start')],
        [types.KeyboardButton(text='/help (в разработке)')]
        ]
        keyboard = types.ReplyKeyboardMarkup(keyboard=kb, resize_keyboard=True, one_time_keyboard=True)
        await message.answer_sticker(r'CAACAgUAAxkBAAEGn99jh1oHZzET1tjBRiN3w_6ru1-TBAAC4QUAAqMFQQdMy6ISqoHknCsE')
        await message.answer('Нельзя просто прийти и что-то в меня кинуть.\nСначала надо поздороваться.')
        await message.answer('Нажмите /start, чтобы начать общение.\n\
Нажмите /help, чтобы узнать обо мне.', reply_markup=keyboard)
    
@router.message(lambda message: message.text == 'Спасибо, не надо облаков.')
async def cancel(message):
    global switch
    switch = 'off'
    await message.answer_sticker(r'CAACAgUAAxkBAAEGli5jhJ3TB9AIa1XroQQlRAlSvbXrBQAC8QUAAqMFQQeTfXHOdOBBJysE')
    await message.answer('Понимаю.\n\nВведите любое сообщение, если захотите делать облака.')
    
@router.message(Command('help'))
async def about(message): 
    global switch
    switch = 'off'
    await message.answer_sticker(r'CAACAgUAAxkBAAEGn-Vjh1qEke0WNzojT5J6kGUttB0idgACxQUAAqMFQQd6VxpupFr34SsE')
    await message.answer('Разрешите представиться.\n\n\
Я облако Клауди и я делаю облака слов на основе файлов, содержащих текст. В данный момент я принимаю на вход:\n\n\
1. JSON файлы с историей (всеми сообщениями) Telegram каналов или чатов, которые можно выгрузить \
из Telegram Desktop (ПК-версия Telegram). Мобильная и браузерная версии Telegram не экспортируют историю переписки.\n\n\
2. Обычные текстовые файлы (блокноты) с разрешением .txt.\n\n\
На выходе Вы получите облако слов (а именно: существительных в начальной форме) в виде изображения с расширением .png.\n\n\
На любом экране нашей переписки можно ввести любое сообщение, даже один символ, чтобы увидеть две кнопки:\n\n\
/start - отправляет Вас в главное меню, состоящее на данный момент из трёх пунктов: 2 метода ввода и отмена;\n\n\
/help - отправляет Вас сюда.\n\n\
Длительность обработки файлов зависит от объёмов текста. Но Вы можете выйти из чата и через некоторое время \
(секунды, минуты, часы...) Вам придёт от меня сообщение с готовым облаком слов...\n\n\
... Если всё получится. Но у нас должно получиться, хоть я и нахожусь на стадии разработки.\n\n\
Желаю Вам приятного облакотворения!')
       
@router.message()
async def every(message):
    global switch
    switch = 'off'
    kb = [
        [types.KeyboardButton(text='/start')],
        [types.KeyboardButton(text='/help (в разработке)')]
    ]
    keyboard = types.ReplyKeyboardMarkup(keyboard=kb, resize_keyboard=True, one_time_keyboard=True)
    await message.answer('Нажмите /start, чтобы начать общение.\n\
Нажмите /help, чтобы узнать обо мне.', reply_markup=keyboard)

if __name__ == "__main__":
    asyncio.run(main())

INFO:aiogram.dispatcher:Start polling
INFO:aiogram.dispatcher:Run polling for bot @very_cloudy_bot id=5930567047 - 'Cloudy'
INFO:aiogram.event:Update id=983441765 is handled. Duration 343 ms by bot id=5930567047
